# Plasma


## Jupyter Notebook de démonstration

Adaptation du TP5 Testis Vs Ovary, fait dans l'UE1 du M1 MEG

Sandrine Caburet, Claire Vandiedonck et Pierre Poulain, novembre 2020

---
Ceci est un jupyter notebook en R, ce qui signifie que les commandes seront des commandes R directement interprétables par Plasma.

__*Utilisation d'un notebook*__

- Pour ajouter une nouvelle cellule, cliquez sur l'icône "+" dans la barre des menus
- Vous pouvez "cliquer-glisser" pour bouger une cellule 
- Vous pouvez choisir le type de cellule dans le petit menu déroulant dans la barre des menus : 
    - 'Code' pour entrer des lignes de commandes à executer 
    - 'Markdown cells' pour ajouter simplement du texte, formatable avec quelques signes 
- Pour executer une cellule 'Code', pressez SHIFT+ENTER ou cliquez sur l'icône "play" 
- Pour afficher une cellule 'Markdown', pressez SHIFT+ENTER ou cliquez sur l'icône "play" 
- Pour modifier une cellule 'Markdown', double-cliquez dessus
---

### I - Préparation de l'espace de travail

Par défaut, le répertoire de travail est d'office attribué au répertoire de l'environnement par Jupyter Hub.
Puis nous devons indiquer à R les paquets à charger qui seront nécessaires pour l'analyse. 


In [ ]:
# Chargement des packages dans l'environnement présent
library(Matrix)
library(lattice)
library(fdrtool)
library(rpart)
library(Biobase)
library(Biostrings)
library(mouse4302cdf)
library(ggplot2)
library(limma)
library(affy)
library(affyPLM)
library(simpleaffy)
library(genefilter)

### II - Analyse de données


Les donnée utilisées ici sont des données d’expression (puces d’expression de type Affymetrix).
Les données sont stockées sur le serveur Plasma

Une analyse classique consiste en :
- lecture des données
- contrôle qualité des données
- passage en log2 et normalisation
- plots des résultats


### 1 - Récupération des données

Les fichiers nécessaires étant déjà présents sur le serveur, on y accède via leur chemin, et les données sont chargées dans l'objet R adéquat.

In [ ]:
# Chemin d'accès aux données sur plasmabio :
celpath = "/srv/data/meg-m1-ue1/DataTP5/"

# Import des données d'intensité de fluorescence brutes des sondes depuis les fichiers CEL, 
# et ajout dans un objet R AffyBatch
data = ReadAffy(celfile.path=celpath)
ph = data@phenoData
ph@data

In [ ]:
#Récupération de l'annotation des sondes
feat = data@featureData
feat
feat@data


### 2 - Contrôle qualité des données


In [ ]:
# Attribution de noms informatifs aux échantillons
ph@data[ ,1] = c("Testis_E13_1","Testis_E13_2","Testis_E13_3","Testis_P0_1","Testis_P0_2","Testis_P0_3","Ovary_E13_1","Ovary_E13_2","Ovary_E13_3", "Ovary_P0_1", "Ovary_P0_2", "Ovary_P0_3")

#### Histogrammes de distribution des données 
Puis nous créons 2 histogrammes de la distribution des données (avec ggplot2).
Le premier affiche la répartition des données brutes, le second celle des valeurs passées en log2.

In [ ]:
# Récupérations des intensités PM (sondes Perfect Match dans les probe sets)
pmexp = pm(data)

# initialisation de 3 vecteurs
sampleNames = vector()
nologs = vector()
logs = vector()

# remplissage des vecteurs avec les données
for (i in 1:12)
{
  sampleNames = c(sampleNames,rep(ph@data[i,1],dim(pmexp)[1]))
  nologs = c(nologs, pmexp[,i])
  logs = c(logs,log2(pmexp[,i]))
}

# Après avoir rempli les vecteurs, nous combinons sampleNames et nologs ou logs dans 2 dataframes
nologData = data.frame(nologInt=nologs,sampleName=sampleNames)
logData = data.frame(logInt=logs,sampleName=sampleNames)

In [ ]:
# Maintenant nous pouvons créer les 2 histogrammes: 

# Decide plots size 
options(repr.plot.width=18, repr.plot.height= 7)

# Histogramme avec les valeurs brutes
ggplot(nologData, aes(nologInt, colour = sampleName)) +
 geom_density() +
 theme_bw(base_size = 20)
   
# Histogramme avec les valeurs en log2
ggplot(logData, aes(logInt, colour = sampleName)) +
 geom_density() +
 theme_bw(base_size = 20)

#### Boxplots de distribution des données 
Des boxplots permettent de comparer la répartition des données entre échantillons.


In [ ]:
options(repr.plot.width=12, repr.plot.height= 7)

ggplot(logData,aes(sampleName,logInt)) + 
 geom_boxplot() +
 theme_grey(base_size = 17)


#### MA plots
Les plots MA permettent de visualiser la variablité de l'expression des gènes, et sa répartition en fonction du niveau d'expression.



In [ ]:
MAplot(data,which=3)

On peut aussi avoir ces plots dans des fichiers séparés, dans la colonne de gauche

In [ ]:
# Création des MA plots pour chaque échantillon, chacun dans un fichier séparé, qui apparait dans la colonne de gauche.

for (i in 4:5)
{
  name = paste("MAplot",i,".jpg",sep="")
  jpeg(name, width=800, height=600)
  MAplot(data,which=i)
  dev.off()
}

### 3 - Normalisation des données
Données normalisées avec la méthode GCRMA, une des plus aboutie actuellement. L'effet de la normalisation se voit grâce aux mêmes plots. 

In [ ]:
# Chargement de l'objet "data_gcrma.RData", pour le réintégrer dans la procédure d'analyse
load("data_gcrma.RData")

# Box plots des valeurs normalisées
# création d'une data frame avec seulement les valeurs d'expression normalisées
normexpr.gcrma = data.frame(exprs(data.gcrma))

# initialisation des vecteurs
sampleNames = vector()
normlogs = vector()

# remplissage des vecteurs
for (i in 1:12)
{
    sampleNames = c(sampleNames,rep(ph@data[i,1],dim(data.gcrma)[1]))
    normlogs = c(normlogs,normexpr.gcrma[,i])
}

# création de la dataframe avec les 2 vecteurs
normData = data.frame(norm_logInt=normlogs,sampleName=sampleNames)


# création des graphs
dataBox3 = ggplot(normData, aes(sampleName,norm_logInt))
dataBox3 + geom_boxplot() + ylim(2,16) + theme_grey(base_size = 17) + ggtitle("After normalization")

MAplot(data.gcrma,which=3)

### 4 - Groupement des échantillons : PCA (Principal Componant Analysis).  



In [ ]:
# Création d'un plot PCA avec ggplot2, à partir des données normalisées
data.PCA = prcomp(t(normexpr.gcrma),scale.=FALSE)

# la part de la variation expliquée par chaque axe est le carré de sdev (standard deviation),
# un des éléments fournis par la fonction prcomp()
pca.var = data.PCA$sdev^2
# et on l'exprime en % du total :
pca.var.per = round(pca.var/sum(pca.var)*100, 1)

# formatage des données
pca.datagg = data.frame(Sample = ph@data[ ,1],
                        X = data.PCA$x[,1],
                        Y = data.PCA$x[,2])
# Make plot wider 
options(repr.plot.width=19, repr.plot.height=11)

# PCA plot avec ggplot2 
ggplot(data = pca.datagg, aes (x=X, y=Y, label = Sample, colour = Sample)) +
  geom_text(size = 6.5) +
  xlab(paste ("PC1 - ", pca.var.per[1], " %", sep = "")) +
  ylab(paste ("PC2 - ", pca.var.per[2], " %", sep = "")) +
  theme_classic(base_size = 20) + 
  theme (legend.position = "none") +
  ggtitle("  PCA plot Testis & Ovary samples")






----

Les éléments chargés dans l'environnement sont indiqués via la commande suivante : 

In [ ]:
sessionInfo()